In [1]:
# Config Data Structure
import IPython
import IPython.display

import pandas as pd
import numpy as np
import pprint as pp
import random as ran
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import random as ran
import math

from numpy import dot
from numpy.linalg import norm
import numpy as np
from scipy.spatial.distance import euclidean as euc

from functools import reduce
from collections import Counter
from datetime import datetime as dt
from pymongo import MongoClient as mc
from functools import reduce

sns.set(rc={'figure.figsize': (15.7, 13.27)})
plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)
keti_db = client.keti_pattern_recognition

jungang_col = keti_db.jungang_pattern
cluster_col = keti_db.cluster_info
weather_col = keti_db.weather_info

In [2]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

def plot(self, model=None, plot_col='kw (15min)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_subplots, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend().remove()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=False,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

MAX_EPOCHS = 100

def compile_and_fit(model, window, EPOCHS = 20,patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[
                    tf.metrics.MeanAbsoluteError(),
                    
                ])

  history = model.fit(window.train, epochs=EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [40]:
jungang_db_cur = jungang_col.find()
db_datas = [_ for _ in jungang_db_cur]

jg_datas = pd.DataFrame(columns=['Date Time','energy (kw 15min)'])
jg_datas['Date Time'] = [_['ttime'] for _ in db_datas]
jg_datas['energy (kw 15min)'] = [_['energy'] for _ in db_datas]

date_time = pd.to_datetime(jg_datas.pop('Date Time'),
                          format="%Y-%m-%d %H:%M:%S")
jg_datas.index = date_time


# 불 필요 데이터 잘라내기
idx = jg_datas.index.get_loc(jg_datas[jg_datas['energy (kw 15min)'] == 0].index[7])

jg_datas = jg_datas.iloc[:idx].copy()
jg_datas

,energy (kw 15min)
Date Time,
2017-01-01 00:00:00,15250
2017-01-01 00:15:00,15250
2017-01-01 00:30:00,13750
2017-01-01 00:45:00,14250
2017-01-01 01:00:00,14000
...,...
2018-09-19 22:45:00,18750
2018-09-19 23:00:00,18250
2018-09-19 23:15:00,18000


In [41]:
def calc_sin(ts, target_value):
    return np.sin(ts * (2 * np.pi / target_value)).values


def calc_cos(ts, target_value):
    return np.cos(ts * (2 * np.pi / target_value)).values

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

def get_season(month):
    if month in [3, 4, 5]:
        return "봄"
    elif month in [6, 7, 8]:
        return "여름"
    elif month in [9, 10, 11]:
        return "가을"
    else:
        return "겨울"


def get_season_from_idx(idx):
    if idx == 1:
        return "봄"
    elif idx == 2:
        return "여름"
    elif idx == 3:
        return "가을"
    elif idx == 4:
        return "겨울"


def get_season_to_idx(season):
    if season == "봄":
        return 1
    elif season == "여름":
        return 2
    elif season == "가을":
        return 3
    elif season == "겨울":
        return 4


SEASONS = ["봄", "여름", "가을", "겨울"]


def get_weather_to_idx(weather):
    if weather == "특이사항없음":
        return 1
    elif weather == "비":
        return 2
    elif weather == "눈":
        return 3

# ~ 2018 year data parsing
jg_datas = jg_datas[jg_datas.index.year <= 2018]
date_time = jg_datas.index
timestamp = date_time.map(dt.timestamp)

day = 24 * 60 * 60
week = 7 * day
year = (365) * day

jg_datas['week sin'] = calc_sin(timestamp, week)
jg_datas['week cos'] = calc_cos(timestamp, week)
jg_datas['year sin'] = calc_sin(timestamp, year)
jg_datas['year cos'] = calc_cos(timestamp, year)
jg_datas['season'] = [get_season(_.month) for _ in jg_datas.index]
jg_datas['season idx'] = [get_season_to_idx(_) for _ in jg_datas['season'].values]

jg_datas = jg_datas[::4]
jg_datas

,energy (kw 15min),week sin,week cos,year sin,year cos,season,season idx
Date Time,,,,,,,
2017-01-01 00:00:00,15250,0.707107,-0.707107,0.198782,0.980044,겨울,4
2017-01-01 01:00:00,14000,0.680173,-0.733052,0.199485,0.979901,겨울,4
2017-01-01 02:00:00,14750,0.652287,-0.757972,0.200188,0.979758,겨울,4
2017-01-01 03:00:00,29500,0.623490,-0.781831,0.200891,0.979614,겨울,4
2017-01-01 04:00:00,15750,0.593820,-0.804598,0.201593,0.979469,겨울,4
...,...,...,...,...,...,...,...
2018-09-19 19:00:00,22500,-0.500000,0.866025,-0.999984,-0.005738,가을,3
2018-09-19 20:00:00,19250,-0.467269,0.884115,-0.999987,-0.005021,가을,3
2018-09-19 21:00:00,20250,-0.433884,0.900969,-0.999991,-0.004304,가을,3


In [42]:
# 날씨 데이터 가져오기
weather_df = pd.DataFrame(columns=['weather', 'avg ta', 'avg rhm'])
for weather in weather_col.find():
    try:
        iscs = weather['weather']
    except:
        iscs = "특이사항없음"
    if iscs != "특이사항없음":
        iscs_idx = 0
        for hours in range(0, 24):
            if (iscs[iscs_idx]['end time'] == ""):
                iscs[iscs_idx]['end time'] = weather['date'].replace(
                    hour=23)
            if (hours >= iscs[iscs_idx]['start time'].hour) and \
                    (hours <= iscs[iscs_idx]['end time'].hour):
                weather_df = weather_df.append({
                    "weather": iscs[iscs_idx]['weather'],
                    "avg ta": weather['avgTa'],
                    "avg rhm": weather['avgRhm']
                }, ignore_index=True)
            else:
                weather_df = weather_df.append({
                    "weather": "특이사항없음",
                    "avg ta": weather['avgTa'],
                    "avg rhm": weather['avgRhm']
                }, ignore_index=True)

            if ((iscs_idx + 1) < len(iscs)) and \
                    (iscs[iscs_idx]['end time'].hour < (hours + 1)):
                iscs_idx += 1
    else:
        for hours in range(0, 24):
            weather_df = weather_df.append({
                "weather": "특이사항없음",
                "avg ta": weather['avgTa'],
                "avg rhm": weather['avgRhm']
            }, ignore_index=True)

all_data_length = len(jg_datas)
jg_datas['weather'] = list(weather_df['weather'].values)[
    :all_data_length]
jg_datas['weather idx'] = list(get_weather_to_idx(
    _) for _ in weather_df['weather'].values)[:all_data_length]
jg_datas['avg ta'] = list(map(float, weather_df['avg ta'].values))[
    :all_data_length]
jg_datas['avg rhm'] = list(map(float, weather_df['avg rhm'].values))[
    :all_data_length]

jg_datas

,energy (kw 15min),week sin,week cos,year sin,year cos,season,season idx,weather,weather idx,avg ta,avg rhm
Date Time,,,,,,,,,,,
2017-01-01 00:00:00,15250,0.707107,-0.707107,0.198782,0.980044,겨울,4,특이사항없음,1,2.7,75.9
2017-01-01 01:00:00,14000,0.680173,-0.733052,0.199485,0.979901,겨울,4,특이사항없음,1,2.7,75.9
2017-01-01 02:00:00,14750,0.652287,-0.757972,0.200188,0.979758,겨울,4,특이사항없음,1,2.7,75.9
2017-01-01 03:00:00,29500,0.623490,-0.781831,0.200891,0.979614,겨울,4,특이사항없음,1,2.7,75.9
2017-01-01 04:00:00,15750,0.593820,-0.804598,0.201593,0.979469,겨울,4,특이사항없음,1,2.7,75.9
...,...,...,...,...,...,...,...,...,...,...,...
2018-09-19 19:00:00,22500,-0.500000,0.866025,-0.999984,-0.005738,가을,3,비,2,22.1,59.1
2018-09-19 20:00:00,19250,-0.467269,0.884115,-0.999987,-0.005021,가을,3,비,2,22.1,59.1
2018-09-19 21:00:00,20250,-0.433884,0.900969,-0.999991,-0.004304,가을,3,비,2,22.1,59.1


In [43]:
all_dict = dict()
train_type = ['univariate', 
              'multivariate', 
#               'season univariate', 
#               'season multivariate', 
              'multivariate (ta-rhm)', 
              'multivariate (week)', 
              'multivariate (weather)',
              'multivariate (season)',              
              'multivariate (week-season)',              
              'with cluster label',
              'with cluster label (multivariate)',
              'with cluster label (multivariate-weather)',
              'with cluster label (multivariate-ta-rhm)',
              'with cluster label (multivariate-week)',
              'with cluster label (multivariate-season)',
              'with cluster label (multivariate-week-season)'              
             ]
uni_columns = ['energy (kw 15min)']
mul_week_columns = ['energy (kw 15min)', 'week sin', 'week cos']
mul_only_season_columns = ['energy (kw 15min)', 'season idx']
mul_weather_columns = ['energy (kw 15min)', 'weather idx']
mul_season_columns = ['energy (kw 15min)', 'week sin', 'week cos', 'season idx']
mul_ta_rhm_columns = ['energy (kw 15min)', 'avg ta', 'avg rhm'] 
mul_columns = ['energy (kw 15min)', 'week sin', 'week cos', 'year sin', 'year cos', 'season idx', 'avg ta', 'avg rhm']
seasons = ["봄", "여름", "가을", "겨울"]

# Clusterinbg 가져오기 작업
cur_cluster_result = cluster_col.find({
    "uid": "jungang_pattern"
})
cluster_result = dict()

for data in cur_cluster_result:
    in_dict = pd.DataFrame(columns=['Label', 'Weekday'])
    in_dict.index.name = "Date Time"
    
    season = data['season']
    infos = data['info']
    
    dtime = [dt.strptime(_['date'], "%Y-%m-%d") for _ in infos]
    labels = [_['label'] for _ in infos]

    for idx, _ in enumerate(dtime):
        label = labels[idx]        
        in_dict.loc[_] = [label, _.weekday()]
        
    cluster_result[season] = in_dict
    
cluster_pattern_dict = dict()

for season in seasons:
    result = cluster_result[season]
    in_dict = pd.DataFrame(columns=[_ for _ in range(0, 24)])
    in_dict.index.name = "Label"
    
    labels = list(set(result['Label']))
    for label in labels:
        cluster_pattern = np.array([])
        date_in_labels = result[result['Label'] == label].index
        for date in date_in_labels:
            idx = jg_datas.index.get_loc(date)
            pattern = jg_datas.iloc[idx: idx+24]['energy (kw 15min)'].values
            cluster_pattern = np.append(cluster_pattern, pattern)
        cluster_pattern = cluster_pattern.reshape(-1,24).mean(axis=0)
        in_dict.loc[label] = cluster_pattern
        
    cluster_pattern_dict[season] = in_dict
    
# Clustering Matching
cluster_dist_dict = dict()
for season in cluster_result.keys():
    cluster_season_dict = dict()
    week_list = set(cluster_result[season]['Weekday'])
    for week in week_list:
        week_dist = cluster_result[season][
            cluster_result[season]['Weekday'] == week
        ]['Weekday'].groupby(cluster_result[season]['Label']).count().sort_values(ascending=False)
        week_top_label = week_dist.index[0]
        
        cluster_season_dict[week] = week_top_label
    cluster_dist_dict[season] = cluster_season_dict

In [44]:
cluster_pattern_dict

{'봄':                  0             1             2             3             4   \
 Label                                                                         
 0      15211.538462  14750.000000  13750.000000  23807.692308  18403.846154   
 1      19000.000000  18500.000000  18343.750000  26187.500000  21906.250000   
 2      14416.666667  13500.000000  13583.333333  19666.666667  15000.000000   
 3      15961.538462  15557.692308  15173.076923  25038.461538  20788.461538   
 4      20720.000000  19510.000000  18990.000000  27010.000000  23830.000000   
 5      21795.454545  20511.363636  20159.090909  26363.636364  24284.090909   
 
                  5             6             7             8             9   \
 Label                                                                         
 0      22942.307692  24884.615385  27557.692308  31596.153846  33326.923077   
 1      29375.000000  29750.000000  32656.250000  36156.250000  38906.250000   
 2      24583.333333  23666.66666

In [160]:
def matching(datas):
#     print(datas)
    
    date = datas.index[0]
    season = get_season(date.month)
    times = [_.hour for _ in datas.index]
    
#     print(date)
#     print(season)
#     print(times)
    
    c_patterns = cluster_pattern_dict[season][times].copy().values
#     print("\n{}".format(c_patterns))
    
    
    og_pattern = datas['energy (kw 15min)'].values
#     print("\n{}".format(og_pattern))
    
    dis_info = pd.DataFrame(columns=["dis", "sim", "label"])
    for idx,_ in enumerate(c_patterns):
        dis = euc(_, og_pattern)
        sim = 1 - cos_sim(_, og_pattern)
        
        dis_info = dis_info.append({
            "dis": dis,
            "sim": sim,
            "label": int(idx)
        }, ignore_index=True)
#     print("\n{}\n".format(dis_info))
    
    label = dis_info.sort_values(by=['dis', 'sim'])['label'].iloc[0]
#     print("{}\n".format(label))
    
    print("sel ==> {} \n".format(c_patterns[int(label)]))
    return c_patterns[int(label)]

In [161]:
matching_result = np.array([])

In [162]:
jump = 3
for idx in range(0,len(jg_datas),jump):
    matching_result = np.append(
        matching_result
        ,matching(jg_datas[idx: idx + jump])
    )

sel ==> [15875.  13875.  14062.5] 

sel ==> [22843.75  19328.125 28562.5  ] 

sel ==> [27953.125 29906.25  31578.125] 

sel ==> [36031.25  36906.25  38390.625] 

sel ==> [34770.83333333 35083.33333333 35750.        ] 

sel ==> [32625.         32229.16666667 33145.83333333] 

sel ==> [39050. 34925. 27450.] 

sel ==> [25775. 22575. 20150.] 

sel ==> [18390.625 17125.    16265.625] 

sel ==> [23640.625 23125.    30406.25 ] 

sel ==> [29913.04347826 34804.34782609 40652.17391304] 

sel ==> [44575. 42050. 48875.] 

sel ==> [44375. 42650. 47725.] 

sel ==> [42675. 42000. 44525.] 

sel ==> [37703.125 36000.    34640.625] 

sel ==> [33010.86956522 23021.73913043 20152.17391304] 

sel ==> [15875.  13875.  14062.5] 

sel ==> [23141.30434783 19847.82608696 28554.34782609] 

sel ==> [27953.125 29906.25  31578.125] 

sel ==> [53065.2173913 54565.2173913 54750.       ] 

sel ==> [55434.7826087  54880.43478261 50728.26086957] 

sel ==> [46554.34782609 47565.2173913  46880.43478261] 

sel ==> [56234.3


sel ==> [23640.625 23125.    30406.25 ] 

sel ==> [29453.125 36625.    40375.   ] 

sel ==> [53890.625 57046.875 61375.   ] 

sel ==> [61140.625 60703.125 58234.375] 

sel ==> [46554.34782609 47565.2173913  46880.43478261] 

sel ==> [56234.375 49875.    41593.75 ] 

sel ==> [37437.5   27812.5   24484.375] 

sel ==> [18978.26086957 17173.91304348 16978.26086957] 

sel ==> [23640.625 23125.    30406.25 ] 

sel ==> [29453.125 36625.    40375.   ] 

sel ==> [53890.625 57046.875 61375.   ] 

sel ==> [55434.7826087  54880.43478261 50728.26086957] 

sel ==> [56000.   54343.75 55343.75] 

sel ==> [49086.95652174 42369.56521739 36597.82608696] 

sel ==> [33000.   23968.75 21593.75] 

sel ==> [18390.625 17125.    16265.625] 

sel ==> [23141.30434783 19847.82608696 28554.34782609] 

sel ==> [29453.125 36625.    40375.   ] 

sel ==> [53065.2173913 54565.2173913 54750.       ] 

sel ==> [55434.7826087  54880.43478261 50728.26086957] 

sel ==> [36734.375 37109.375 34890.625] 

sel ==> [49086.956521

sel ==> [37437.5   27812.5   24484.375] 

sel ==> [21437.5   20234.375 19078.125] 

sel ==> [18575. 18650. 20150.] 

sel ==> [27953.125 29906.25  31578.125] 

sel ==> [36031.25  36906.25  38390.625] 

sel ==> [38515.625 37609.375 36718.75 ] 

sel ==> [36734.375 37109.375 34890.625] 

sel ==> [37703.125 36000.    34640.625] 

sel ==> [37437.5   27812.5   24484.375] 

sel ==> [21437.5   20234.375 19078.125] 

sel ==> [22843.75  19328.125 28562.5  ] 

sel ==> [27953.125 29906.25  31578.125] 

sel ==> [36031.25  36906.25  38390.625] 

sel ==> [38515.625 37609.375 36718.75 ] 

sel ==> [42675. 42000. 44525.] 

sel ==> [37703.125 36000.    34640.625] 

sel ==> [37437.5   27812.5   24484.375] 

sel ==> [21437.5   20234.375 19078.125] 

sel ==> [23640.625 23125.    30406.25 ] 

sel ==> [29453.125 36625.    40375.   ] 

sel ==> [53065.2173913 54565.2173913 54750.       ] 

sel ==> [55434.7826087  54880.43478261 50728.26086957] 

sel ==> [46554.34782609 47565.2173913  46880.43478261] 

sel ==> [5


sel ==> [54636.36363636 55022.72727273 51340.90909091] 

sel ==> [48784.09090909 50090.90909091 48784.09090909] 

sel ==> [39890. 36650. 33800.] 

sel ==> [30153.84615385 22788.46153846 20134.61538462] 

sel ==> [15961.53846154 15557.69230769 15173.07692308] 

sel ==> [26363.63636364 24284.09090909 31079.54545455] 

sel ==> [31340. 35160. 40100.] 

sel ==> [44760. 46440. 48920.] 

sel ==> [54636.36363636 55022.72727273 51340.90909091] 

sel ==> [41760. 41540. 41860.] 

sel ==> [47181.81818182 43806.81818182 38704.54545455] 

sel ==> [37102.27272727 28159.09090909 24556.81818182] 

sel ==> [21795.45454545 20511.36363636 20159.09090909] 

sel ==> [27010. 23830. 29810.] 

sel ==> [32965.90909091 39056.81818182 45886.36363636] 

sel ==> [52829.54545455 53625.         54136.36363636] 

sel ==> [54636.36363636 55022.72727273 51340.90909091] 

sel ==> [41760. 41540. 41860.] 

sel ==> [38423.07692308 35019.23076923 31250.        ] 

sel ==> [30153.84615385 22788.46153846 20134.61538462] 

sel


sel ==> [31340. 35160. 40100.] 

sel ==> [44760. 46440. 48920.] 

sel ==> [49470. 48390. 43960.] 

sel ==> [41760. 41540. 41860.] 

sel ==> [47181.81818182 43806.81818182 38704.54545455] 

sel ==> [37102.27272727 28159.09090909 24556.81818182] 

sel ==> [20720. 19510. 18990.] 

sel ==> [23807.69230769 18403.84615385 22942.30769231] 

sel ==> [26288.46153846 30403.84615385 34211.53846154] 

sel ==> [33326.92307692 35211.53846154 35903.84615385] 

sel ==> [35000.         34038.46153846 32269.23076923] 

sel ==> [30173.07692308 30673.07692308 29730.76923077] 

sel ==> [29980.76923077 27826.92307692 26673.07692308] 

sel ==> [30153.84615385 22788.46153846 20134.61538462] 

sel ==> [15211.53846154 14750.         13750.        ] 

sel ==> [23807.69230769 18403.84615385 22942.30769231] 

sel ==> [24884.61538462 27557.69230769 31596.15384615] 

sel ==> [33326.92307692 35211.53846154 35903.84615385] 

sel ==> [35000.         34038.46153846 32269.23076923] 

sel ==> [30173.07692308 30673.076923

...


sel ==> [ 9500.         10208.33333333 12750.        ] 

sel ==> [13541.66666667 13291.66666667 10375.        ] 

sel ==> [9666.66666667 8708.33333333 8708.33333333] 

sel ==> [8291.66666667 8125.         8583.33333333] 

sel ==> [12328.125 11453.125 10796.875] 

sel ==> [10218.75  10125.     9953.125] 

sel ==> [9453.125 9218.75  9843.75 ] 

sel ==> [12093.75  12640.625 13531.25 ] 

sel ==> [26750. 30500. 35000.] 

sel ==> [34296.875 34781.25  33062.5  ] 

sel ==> [31087.5 31550.  30762.5] 

sel ==> [31912.5 25837.5 21887.5] 

sel ==> [21025.  19187.5 18600. ] 

sel ==> [15300.  14887.5 14475. ] 

sel ==> [13187.5 12887.5 13737.5] 

sel ==> [14887.5 18662.5 22700. ] 

sel ==> [29425.  30725.  34062.5] 

sel ==> [37275.  36837.5 33300. ] 

sel ==> [37416.66666667 36750.         37500.        ] 

sel ==> [31912.5 25837.5 21887.5] 

sel ==> [21025.  19187.5 18600. ] 

sel ==> [15300.  14887.5 14475. ] 

sel ==> [13187.5 12887.5 13737.5] 

sel ==> [11583.33333333 14916.66666667 19916.666


sel ==> [40083.33333333 42333.33333333 40000.        ] 

sel ==> [37416.66666667 36750.         37500.        ] 

sel ==> [39166.66666667 30250.         19416.66666667] 

sel ==> [21025.  19187.5 18600. ] 

sel ==> [15300.  14887.5 14475. ] 

sel ==> [13187.5 12887.5 13737.5] 

sel ==> [14887.5 18662.5 22700. ] 

sel ==> [29425.  30725.  34062.5] 

sel ==> [40083.33333333 42333.33333333 40000.        ] 

sel ==> [37416.66666667 36750.         37500.        ] 

sel ==> [39166.66666667 30250.         19416.66666667] 

sel ==> [21025.  19187.5 18600. ] 

sel ==> [15300.  14887.5 14475. ] 

sel ==> [13187.5 12887.5 13737.5] 

sel ==> [14887.5 18662.5 22700. ] 

sel ==> [26750. 30500. 35000.] 

sel ==> [34296.875 34781.25  33062.5  ] 

sel ==> [22826.08695652 22184.7826087  21576.08695652] 

sel ==> [20695.65217391 15923.91304348 12543.47826087] 

sel ==> [21025.  19187.5 18600. ] 

sel ==> [15300.  14887.5 14475. ] 

sel ==> [13187.5 12887.5 13737.5] 

sel ==> [14887.5 18662.5 22700. ] 



sel ==> [13453.125 13171.875 13921.875] 

sel ==> [13593.75 13312.5  13031.25] 

sel ==> [13500.    13203.125 12593.75 ] 

sel ==> [13187.5 12887.5 13737.5] 

sel ==> [14887.5 18662.5 22700. ] 

sel ==> [29425.  30725.  34062.5] 

sel ==> [40083.33333333 42333.33333333 40000.        ] 

sel ==> [37416.66666667 36750.         37500.        ] 

sel ==> [39166.66666667 30250.         19416.66666667] 

sel ==> [21025.  19187.5 18600. ] 

sel ==> [15300.  14887.5 14475. ] 

sel ==> [13187.5 12887.5 13737.5] 

sel ==> [14887.5 18662.5 22700. ] 

sel ==> [29425.  30725.  34062.5] 

sel ==> [40083.33333333 42333.33333333 40000.        ] 

sel ==> [37416.66666667 36750.         37500.        ] 

sel ==> [31912.5 25837.5 21887.5] 

sel ==> [17083.33333333 15750.         14166.66666667] 

sel ==> [15300.  14887.5 14475. ] 

sel ==> [13187.5 12887.5 13737.5] 

sel ==> [14887.5 18662.5 22700. ] 

sel ==> [29425.  30725.  34062.5] 

sel ==> [40083.33333333 42333.33333333 40000.        ] 

sel ==> [3


sel ==> [39166.66666667 30250.         19416.66666667] 

sel ==> [21025.  19187.5 18600. ] 

sel ==> [15300.  14887.5 14475. ] 

sel ==> [13187.5 12887.5 13737.5] 

sel ==> [14887.5 18662.5 22700. ] 

sel ==> [29425.  30725.  34062.5] 

sel ==> [40083.33333333 42333.33333333 40000.        ] 

sel ==> [37416.66666667 36750.         37500.        ] 

sel ==> [31912.5 25837.5 21887.5] 

sel ==> [17083.33333333 15750.         14166.66666667] 

sel ==> [15300.  14887.5 14475. ] 

sel ==> [13187.5 12887.5 13737.5] 

sel ==> [14887.5 18662.5 22700. ] 

sel ==> [27656.25  29656.25  32328.125] 

sel ==> [27347.82608696 27663.04347826 24815.2173913 ] 

sel ==> [13765.625 13796.875 13484.375] 

sel ==> [13453.125 13171.875 13921.875] 

sel ==> [11847.82608696 10826.08695652 10565.2173913 ] 

sel ==> [10218.75  10125.     9953.125] 

sel ==> [11859.375 11687.5   11968.75 ] 

sel ==> [12093.75  12640.625 13531.25 ] 

sel ==> [ 9500.         10208.33333333 12750.        ] 

sel ==> [13541.66666667 

sel ==> [35083.33333333 34958.33333333 37416.66666667] 

sel ==> [37250.         31291.66666667 24833.33333333] 

sel ==> [22083.33333333 20250.         19041.66666667] 

sel ==> [17500.         17791.66666667 16916.66666667] 

sel ==> [17041.66666667 17375.         17208.33333333] 

sel ==> [17291.66666667 24708.33333333 31416.66666667] 

sel ==> [25875.         28875.         30083.33333333] 

sel ==> [29395.83333333 30625.         27583.33333333] 

sel ==> [22777.77777778 23236.11111111 23861.11111111] 

sel ==> [28309.52380952 25357.14285714 19761.9047619 ] 

sel ==> [22083.33333333 20250.         19041.66666667] 

sel ==> [17500.         17791.66666667 16916.66666667] 

sel ==> [17041.66666667 17375.         17208.33333333] 

sel ==> [17291.66666667 24708.33333333 31416.66666667] 

sel ==> [20977.27272727 22090.90909091 23636.36363636] 

sel ==> [22477.27272727 23068.18181818 21113.63636364] 

sel ==> [22777.77777778 23236.11111111 23861.11111111] 

sel ==> [28309.52380952 25357.1

In [164]:
len(matching_result)

15048

In [167]:
cluster_pattern_col = pd.DataFrame(columns=['cluster energy'])

In [169]:
cluster_pattern_col['cluster energy'] = matching_result

In [170]:
cluster_pattern_col

,cluster energy
0,15875.000000
1,13875.000000
2,14062.500000
3,22843.750000
4,19328.125000
...,...
15043,25357.142857
15044,19761.904762
15045,22083.333333
15046,20250.000000
